In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import main

In [5]:
## Start of Notebook

print('Starting Assignment 1')
RawData = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
print(RawData.shape)
print(RawData.describe())

## Might not need this line here. Just a precaution for NaN.
# if trainingData.isnull().any:
#     trainingData = trainingData.fillna(method='ffill')


Starting Assignment 1
(111993, 12)
            Instance  Year of Record            Age  Size of City  \
count  111993.000000   111552.000000  111499.000000  1.119930e+05   
mean    55997.000000     1999.421274      37.345304  8.388538e+05   
std     32329.738686       11.576382      16.036694  2.196879e+06   
min         1.000000     1980.000000      14.000000  7.700000e+01   
25%     27999.000000     1989.000000      24.000000  7.273400e+04   
50%     55997.000000     1999.000000      35.000000  5.060920e+05   
75%     83995.000000     2009.000000      48.000000  1.184501e+06   
max    111993.000000     2019.000000     115.000000  4.999251e+07   

       Wears Glasses  Body Height [cm]  Income in EUR  
count  111993.000000     111993.000000   1.119930e+05  
mean        0.500531        175.220192   1.092138e+05  
std         0.500002         19.913889   1.498024e+05  
min         0.000000         94.000000  -5.696906e+03  
25%         0.000000        160.000000   3.077169e+04  
50%    

In [6]:
## Dropping Features that are missing a large amount of data
## Dropping instances with missing data

RawData = main.HandleMissingData(RawData)

null_counts = RawData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

37
1999.4212743832472
Number of null values in each column:
Year of Record      0
Age                 0
Country             0
Size of City        0
Profession          0
Wears Glasses       0
Body Height [cm]    0
Income in EUR       0
dtype: int64


In [7]:
## Checking data for missing values

null_counts = RawData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))


Number of null values in each column:
Year of Record      0
Age                 0
Country             0
Size of City        0
Profession          0
Wears Glasses       0
Body Height [cm]    0
Income in EUR       0
dtype: int64


In [8]:
## Seeing the data and the unique values

for s in RawData.columns.values:
    print("Features in " + s)
    print(RawData[s].unique())
    print(len(RawData[s].unique()))


Features in Year of Record
[1997. 1996. 2018. 2006. 2010. 1991. 1982. 2008. 2015. 2019. 1981. 1989.
 2003. 1987. 1995. 1992. 1980. 1983. 2011. 2014. 1985. 2005. 1998. 2001.
 2017. 2007. 1988. 1994. 2012. 2009. 2013. 2004. 1999. 2002. 1993. 2000.
 1990. 2016. 1986. 1984.]
40
Features in Age
[ 41.  28.  33.  46.  60.  71.  36.  43.  16.  51.  15.  22.  32.  40.
  31.  27.  25.  35.  63.  38.  34.  20.  23.  14.  37.  62.  52.  47.
  24.  26.  50.  49.  18.  19.  39.  66.  21.  44.  29.  64.  65.  53.
  78.  42.  58.  17.  30.  79.  48.  56.  83.  54.  74.  87.  90.  67.
  45.  77.  73.  55.  92.  80.  72.  57.  70.  59.  61.  82.  69.  98.
  94.  68.  81.  97.  76.  75.  88.  91.  96.  84.  89. 107.  85.  86.
 100. 110.  95.  99.  93. 104. 102. 101. 105. 111. 108. 103. 109. 106.
 115.]
99
Features in Country
['Belarus' 'Singapore' 'Norway' 'Cuba' 'United Arab Emirates' 'Liberia'
 'State of Palestine' 'Israel' 'South Sudan' 'Kyrgyzstan' 'Togo' 'Finland'
 'Sierra Leone' 'Papua New Guinea' 

In [9]:
## Section to conver to binary matrix
## Changing Country, Profession

RawData_Country = pd.get_dummies(RawData.Country)
print(type(RawData_Country))
RawData_Profession = pd.get_dummies(RawData.Profession)
print(type(RawData_Profession))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [10]:
RawData.describe()

,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR
count,111993.000000,111993.000000,1.119930e+05,111993.000000,111993.000000,1.119930e+05
mean,1999.419616,37.343780,8.388538e+05,0.500531,175.220192,1.092138e+05
std,11.553597,16.001302,2.196879e+06,0.500002,19.913889,1.498024e+05
min,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03
25%,1989.000000,24.000000,7.273400e+04,0.000000,160.000000,3.077169e+04
50%,1999.000000,35.000000,5.060920e+05,1.000000,174.000000,5.733917e+04
75%,2009.000000,48.000000,1.184501e+06,1.000000,190.000000,1.260936e+05
max,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06


In [11]:
## Creating clean training data
print(RawData.columns)

RawData = pd.concat([RawData, RawData_Country], axis=1)
RawData= RawData.drop('Country', axis=1)
RawData = pd.concat([RawData, RawData_Profession], axis=1)
RawData= RawData.drop('Profession', axis=1)

Index(['Year of Record', 'Age', 'Country', 'Size of City', 'Profession',
       'Wears Glasses', 'Body Height [cm]', 'Income in EUR'],
      dtype='object')


In [17]:
RawData.describe()

,Instance,Year of Record,Age,Size of City,Wears Glasses,Body Height [cm],Income in EUR,Afghanistan,Albania,Algeria,...,windows administrator,wireless coordinator,woodworker,word processor,workforce management analyst,workforce planning intern,writer,x-ray technician,yardmaster,youth initiatives lead advisor
count,110739.000000,110739.000000,110739.000000,1.107390e+05,110739.000000,110739.000000,1.107390e+05,110739.00000,110739.000000,110739.000000,...,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000,110739.000000
mean,55977.659343,1999.412790,37.341154,8.378984e+05,0.500673,175.222352,1.090052e+05,0.00019,0.003558,0.000063,...,0.000244,0.000244,0.000307,0.000253,0.000280,0.000217,0.000289,0.000253,0.000181,0.000289
std,32331.256601,11.577877,16.033197,2.187687e+06,0.500002,19.918049,1.493287e+05,0.01377,0.059542,0.007950,...,0.015613,0.015613,0.017520,0.015899,0.016729,0.014720,0.016997,0.015899,0.013438,0.016997
min,1.000000,1980.000000,14.000000,7.700000e+01,0.000000,94.000000,-5.696906e+03,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,27978.500000,1989.000000,24.000000,7.274450e+04,0.000000,160.000000,3.074007e+04,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,55973.000000,1999.000000,35.000000,5.060690e+05,1.000000,174.000000,5.728304e+04,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,83984.500000,2009.000000,48.000000,1.184484e+06,1.000000,190.000000,1.258811e+05,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,111993.000000,2019.000000,115.000000,4.999251e+07,1.000000,265.000000,5.285252e+06,1.00000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Output Training Data to new file
RawData.to_csv("../data/processedData.csv", index=False)

In [ ]:
## Read in new data and model

CleanData = main.ReadInData("../data/processedData.csv")

In [ ]:
CleanData.head(5)

In [ ]:
income = CleanData['Income in EUR']
features = CleanData.drop(labels=['Income in EUR', 'Instance'], axis=1)

features.head(5)

In [ ]:
## Splitting Data

features_train, features_val, income_train, income_val = train_test_split(features, income, test_size=0.2, random_state=0)


In [ ]:
## Using Linear Regression

model = LinearRegression()
model.fit(features_train, income_train)

print(model.coef_)
print(model.intercept_)


In [ ]:
## Predict with the validation data

income_predict = model.predict(features_val)
comparison = pd.DataFrame({'Actual': income_val, 'Predicted': income_predict})

comparison

In [ ]:
## Graphing Difference 

df1 = comparison.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(income_val, income_predict))  
print('Mean Squared Error:', metrics.mean_squared_error(income_val, income_predict))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(income_val, income_predict)))

## Mean Absolute Error: 43557.00458896904
# Mean Squared Error: 7422065078.367375
# Root Mean Squared Error: 86151.40787223025

In [ ]:
## Read in Test and try to map

TestData = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
TestData.head(5)


In [ ]:
## Comparing Training and Test (Country)
print(TestData['Country'].unique())
print("Test Shape " + str(len(TestData['Country'].unique())))
Raw = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
print(Raw['Country'].unique())
print("Raw Shape " + str(len(Raw['Country'].unique())))

print(TestData['Country'].equals(Raw['Country']))

## Test has less than 3 to Training

In [ ]:
## Comparing Training and Test (Profession)
print(TestData['Profession'].unique())
print("Test Shape " + str(len(TestData['Profession'].unique())))
print(Raw['Profession'].unique())
print("Raw Shape " + str(len(Raw['Profession'].unique())))

print(TestData['Profession'].equals(Raw['Profession']))

## Test Data has less 15 to Training

In [ ]:
## Comparing Training and Test (University Degree)
print(TestData['University Degree'].unique())
print("Test Shape " + str(len(TestData['University Degree'].unique())))
print(Raw['University Degree'].unique())
print("Raw Shape " + str(len(Raw['University Degree'].unique())))

print(TestData['University Degree'].equals(Raw['University Degree']))

## Test Data has less 15 to Training

In [ ]:
## Cell to Compare what is in Test and Trainning Data

Diff = pd.DataFrame(columns=['Unique_C_InTest', 'Unique_C_InTrain','Unique_P_InTest', 'Unique_P_InTrain'])

Diff = pd.DataFrame()
Diff['Unique_C_InTest'] = TestData["Country"][~TestData["Country"].isin(Raw["Country"])].drop_duplicates()
print(Diff['Unique_C_InTest'])
print(str(len(Diff['Unique_C_InTest'])))

Diff = pd.DataFrame()
Diff['Unique_P_InTest',] = TestData["Profession"][~TestData["Profession"].isin(Raw["Profession"])].drop_duplicates()
print(Diff['Unique_P_InTest',])
print(str(len(Diff['Unique_P_InTest',])))

Diff = pd.DataFrame()
Diff['Unique_C_InTrain'] = Raw["Country"][~Raw["Country"].isin(TestData["Country"])].drop_duplicates()
print(Diff['Unique_C_InTrain'])
print(str(len(Diff['Unique_C_InTrain'])))

Diff = pd.DataFrame()
Diff['Unique_P_InTrain'] = Raw["Profession"][~Raw["Profession"].isin(TestData["Profession"])].drop_duplicates()
print(Diff['Unique_P_InTrain'])
print(str(len(Diff['Unique_P_InTrain'])))


In [ ]:
Raw.columns.values

In [ ]:
Raw = main.HandleMissingData(Raw)

In [ ]:
null_counts = Raw.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

In [ ]:
TestData = main.HandleMissingData(TestData)
null_counts = TestData.isnull().sum()
print("Number of null values in each column:\n{}".format(null_counts))

In [18]:
## Cell used to test PrepTestForModel

train = main.ReadInData("../data/processedData2.csv")
Raw = main.ReadInData("../data/tcd ml 2019-20 income prediction training (with labels).csv")
test = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
    

In [ ]:
## Cell used to test PrepTestForModel
test = main.ReadInData("../data/tcd ml 2019-20 income prediction test (without labels).csv")
test = main.HandleMissingData(test)
CleanTest = test.drop('Country', axis=1)
CleanTest = CleanTest.drop('Profession', axis=1)
CleanTest = CleanTest.drop('Income', axis=1)

# Create 1st DF as copy. Create 2nd DF with encoding. Cat together

print(CleanTest.columns.unique)
print(train.columns.unique)
countries = Raw['Country'].unique()
professions = Raw['Profession'].unique()

countryDf = pd.DataFrame(columns=countries)
professionDf = pd.DataFrame(columns=professions)

count = 0
series = [0] * len(countries)
    
for i in test['Country']:
    countryDf.loc[count] = series
    if i in countries:
        # Write output
        # "I think this is right... it is right"
        countryDf.loc[count][i] = 1
        
    count = count + 1
    

print(countryDf)

In [26]:
results = np.where(countries == 'Portugal')
print(results)
print(countries[22])

(array([22]),)
Portugal


In [39]:
print(countryDf)

      Belarus Singapore Norway Cuba United Arab Emirates Liberia  \
0           0         0      0    0                    0       0   
1           0         0      0    0                    0       0   
2           0         0      0    0                    0       0   
3           0         0      0    0                    0       0   
4           0         0      0    0                    0       0   
...       ...       ...    ...  ...                  ...     ...   
17048       0         0      0    0                    0       0   
17049       0         0      0    0                    0       0   
17050       0         0      0    0                    0       0   
17051       0         0      0    0                    0       0   
17052       0         0      0    0                    0       0   

      State of Palestine Israel South Sudan Kyrgyzstan  ... Micronesia  \
0                      0      0           0          0  ...          0   
1                      0      0    